In [1]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv
load_dotenv()

BASE_URL = 'https://admin.yelo.red/api/themes'
HEADERS = {
    'sec-ch-ua-platform': '"macOS"',
    'Referer': 'https://admin.yelo.red/en/dashboard/dynamic-themes/edit/8231?name=Freshgo%20AI',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36',
    'Accept': 'application/json, text/plain, */*',
    'sec-ch-ua': '"Chromium";v="134", "Not:A-Brand";v="24", "Google Chrome";v="134"',
    'Content-Type': 'application/json',
    'sec-ch-ua-mobile': '?0',
    'api-key': os.getenv('API_KEY'),
}

ACCESS_TOKEN = os.getenv('ACCESS_TOKEN')
MARKETPLACE_USER_ID = 1703269

In [2]:
def fetch_pages():
    response = requests.post(f"{BASE_URL}/getPagesAndComponents", headers=HEADERS, json={
        "access_token": ACCESS_TOKEN,
        "theme_id": "8231",
        "marketplace_user_id": MARKETPLACE_USER_ID
    })
    response.raise_for_status()
    data = response.json().get("data", {})
    
    pages = data.get("pages", [])
    df_pages = pd.DataFrame([{  
        "page_name": page.get("page_name"),
        "html": page.get("html"),
        "css": page.get("css")
    } for page in pages])
    
    df_pages.to_csv("data/pages_data.csv", index=False)
    print("Pages data saved to pages_data.csv")
    return df_pages

In [3]:
def fetch_components():
    response = requests.post(f"{BASE_URL}/getPagesAndComponents", headers=HEADERS, json={
        "access_token": ACCESS_TOKEN,
        "theme_id": "8231",
        "marketplace_user_id": MARKETPLACE_USER_ID
    })
    response.raise_for_status()
    data = response.json().get("data", {})
    
    template_data = [{
        "template_id": comp.get("template_id"),
        "template_name": comp.get("template_name")
    } for comp in data.get("components", [])]
    
    component_list = []
    for template in template_data:
        comp_response = requests.post("https://beta-api.yelo.red/themes/getTemplate", headers=HEADERS, json={
            "template_id": template["template_id"],
            "access_token": ACCESS_TOKEN,
            "marketplace_user_id": MARKETPLACE_USER_ID
        })
        comp_response.raise_for_status()
        comp_data = comp_response.json().get("data", {}).get("template", {})
        
        component_list.append({
            "template_id": comp_data.get("template_id"),
            "template_name": template["template_name"],
            "html": comp_data.get("html"),
            "css": comp_data.get("css")
        })
    
    df_components = pd.DataFrame(component_list).drop_duplicates(subset=['template_id'])
    df_components.to_csv("data/components_data.csv", index=False)
    print("Components data saved to components_data.csv")
    return df_components

In [4]:
if __name__ == "__main__":
    df_pages = fetch_pages()
    df_components = fetch_components()

Pages data saved to pages_data.csv
Components data saved to components_data.csv
